Assignment 14

In [11]:
import numpy as np
import cv2
import math
import sys
import time


# Get the risk area
def GetRiskAreaImage(dataset):
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()
    # strip the new line character
    data = [x.strip() for x in data]

    # for each character in the data get int value
    for i in range(len(data)):
        data[i] = [int(x) for x in data[i]]

    return np.array(data, dtype=np.uint8)

def GetRisk(map, node):
    return map[node.x,node.y]

class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __repr__(self):
        return 'Node({}, {})'.format(self.x, self.y)
class Route:
    def __init__(self, nodes, risk):
        self.nodes = [x for x in nodes]
        self.risk = risk
    def AddNode(self, node):
        self.nodes.append(node)
    def Contains(self, node):
        return node in self.nodes
    def __repr__(self):
        return 'Route({}, {})'.format(self.nodes, self.risk)

def GetAdjacentNodes(map, current):
    adjacentNodes = []
    adjacentNodes.append(Node(current.x, current.y - 1))
    adjacentNodes.append(Node(current.x, current.y + 1))
    adjacentNodes.append(Node(current.x - 1, current.y))
    adjacentNodes.append(Node(current.x + 1, current.y))

    # Remove Nodes that are not in the map
    adjacentNodes = [x for x in adjacentNodes if x.x >= 0 and x.x < map.shape[0] and x.y >= 0 and x.y < map.shape[1]]
    return adjacentNodes

def showScaledImage(image, breaktime=0, scaling = 1, name='image'):
    Image = np.zeros((image.shape[0]*scaling,image.shape[1]*scaling), dtype=np.uint8)
    for y in range(image.shape[0]): 
        for x in range(image.shape[1]):
            if image[y][x] == 1:
                Image[y*scaling:(y+1)*scaling, x*scaling:(x+1)*scaling] = 255
    #cv2.imwrite(f'./images/aoc11_[{i}].bmp', Image)
    cv2.imshow(name,Image)
    cv2.waitKey(breaktime)
def showWriteImage(image, scaling = 1, loc= ''):
    Image = np.zeros((image.shape[0]*scaling,image.shape[1]*scaling), dtype=np.uint8)
    for y in range(image.shape[0]): 
        for x in range(image.shape[1]):
            if image[y][x] == 1:
                Image[y*scaling:(y+1)*scaling, x*scaling:(x+1)*scaling] = 255
    cv2.imwrite(loc, Image)

# Recursive function that will return the path of lowest risk. 
def FindPathWithLowestRisk( map, current, end,  currentRoute, pathTree, exploredNodesMap, optimalPath = None):
    if current.x < 0 or current.x >= map.shape[0] or current.y < 0 or current.y >= map.shape[1]:
        return None
    if current == end:
        return currentRoute
    else:
        

        # Get the adjacent nodes
        adjacentNodes = GetAdjacentNodes(map, current)

        # remove nodes that are already explored
        adjacentNodes = [x for x in adjacentNodes if exploredNodesMap[x.x][x.y] == 0]
        
        # sort the adjacent nodes by distance to the end
        adjacentNodes = sorted(adjacentNodes, key=lambda x: GetRisk(map, x))

        #print(adjacentNodes)

        for node in adjacentNodes:
            newCurrentRoute = Route(currentRoute.nodes, currentRoute.risk)
            newCurrentRoute.AddNode(node)
            newCurrentRoute.risk += GetRisk(map, node)
            #pathTree.append(newCurrentRoute)
            newExploredNodesMap = np.copy(exploredNodesMap)
            newExploredNodesMap[node.x][node.y] = 1
            if None != optimalPath and newCurrentRoute.risk > optimalPath.risk:
                continue
            showScaledImage(newExploredNodesMap, breaktime=1, scaling = 5, name='exploredNodesMap')
            

            foundRoute = FindPathWithLowestRisk(map, newCurrentRoute.nodes[-1], end, newCurrentRoute, pathTree, newExploredNodesMap, optimalPath)
            
            if foundRoute != None:
                if optimalPath == None or (optimalPath.risk > foundRoute.risk and foundRoute.nodes[-1] == end):
                    optimalPath = foundRoute
                    showScaledImage(MarkRoute(map.shape, optimalPath), breaktime=1, scaling = 5, name='optimalPath')
        return optimalPath    


def FindPathWithLowestRisk2( map, current, end, pathTree,exploredNodesMap, optimalPath = None):
    optimumPathImage = np.zeros(map.shape, dtype=np.uint8)
    while len(pathTree) > 0:    
        #print(f"pathTree: {len(pathTree)} node: {current}")    
        #pathTree.sort( key=lambda x: (x.nodes[-1].x - end.x)**2+ (x.nodes[-1].y - end.y)**2)        
        currentRoute = pathTree.pop(0)
        current = currentRoute.nodes[-1]
        if current.x < 0 or current.x >= map.shape[0] or current.y < 0 or current.y >= map.shape[1]:
            continue
        
        if currentRoute.risk > exploredNodesMap[current.x,current.y] and exploredNodesMap[current.x,current.y] != -1:
            continue

        if optimumPathImage[current.x,current.y] >0:
            collect = False
            for i in range(0,len(optimalPath.nodes)):
                if collect == True:
                    currentRoute.AddNode(optimalPath.nodes[i])
                    currentRoute.risk += GetRisk(map,optimalPath.nodes[i])
                    exploredNodesMap[current.x,current.y] = currentRoute.risk
                elif current == optimalPath.nodes[i]:
                    collect = True
            if (optimalPath.risk > currentRoute.risk and currentRoute.nodes[-1] == end):
                optimumPathImage = MarkRoute(map.shape, currentRoute)
                optimalPath = currentRoute
                showScaledImage(optimumPathImage, breaktime=1, scaling = 5, name='optimalPath')
                continue

        #showScaledImage( MarkRoute(map.shape,currentRoute) , breaktime=1, scaling = 5, name='exploredNodesMap')
        
        # Get the adjacent nodes
        adjacentNodes = GetAdjacentNodes(map, current)


        adjacentNodes = sorted(adjacentNodes, key=lambda x: abs(x.x - end.x)+ abs(x.y - end.y), reverse=True)
        #print(adjacentNodes)
        for node in adjacentNodes:
            newCurrentRoute = Route(currentRoute.nodes, currentRoute.risk)
            newCurrentRoute.risk += GetRisk(map, node)
            newCurrentRoute.AddNode(node)
            if exploredNodesMap[node.x,node.y] > newCurrentRoute.risk or exploredNodesMap[node.x,node.y] == -1:
                exploredNodesMap[node.x,node.y] = newCurrentRoute.risk
                for route in pathTree:
                    if route.Contains(node):
                        pathTree.remove(route)
                #print("adding route")
                pathTree.append(newCurrentRoute)

        if current == end:
            if optimalPath == None or (optimalPath.risk > currentRoute.risk and currentRoute.nodes[-1] == end):
                optimumPathImage = MarkRoute(map.shape, currentRoute)
                optimalPath = currentRoute
                showScaledImage(optimumPathImage, breaktime=1, scaling = 1, name='optimalPath')
            
    return optimalPath      
        
                
            

def MarkRoute(shape,route):
    map = np.zeros(shape, dtype=np.uint8)
    for node in route.nodes:
        map[node.x][node.y] = 1
    return map





In [12]:
if(False):
    map = GetRiskAreaImage('./data/aoc15_test1.txt')

    startRoute = Route([Node(0,0)], 0)
    explored = np.zeros(map.shape, dtype=np.int32)
    explored += sys.maxsize
    print(explored)
    explored[0][0] = 0
    route = FindPathWithLowestRisk2(map, Node(0,0), Node(map.shape[0]-1,map.shape[1]-1), [startRoute], explored)

    MarkedRoute = MarkRoute(map.shape, route)
    print(f'Risk: {route.risk}')
    showScaledImage(MarkedRoute, breaktime=10000, scaling = 20, name= "Result")

    

In [13]:
if(False):
    map = GetRiskAreaImage('./data/aoc15.txt')

    startRoute = Route([Node(0,0)], 0)
    explored = np.zeros(map.shape, dtype=np.int32)
    explored += sys.maxsize
    explored[0][0] = 0
    route = FindPathWithLowestRisk2(map, Node(0,0), Node(map.shape[0]-1,map.shape[1]-1) , [startRoute], explored)

    MarkedRoute = MarkRoute(map.shape, route)

    print(f'Risk: {route.risk}')
    showScaledImage(explored, breaktime=10, scaling = 20)
    showScaledImage(MarkedRoute, breaktime=10, scaling = 20)

In [14]:
def GetRiskAreaImage2(dataset, scaler = 1):
    data = []
    with open(dataset, 'r') as f:
        data = f.readlines()
    # strip the new line character
    data = [x.strip() for x in data]

    # for each character in the data get int value
    for i in range(len(data)):
        data[i] = [int(x) for x in data[i]]
    tmp = np.array(data, dtype=np.uint8)

    tmp = np.zeros((tmp.shape[0]*scaler, tmp.shape[0]*scaler), dtype=np.uint8)

    for i in range(len(data)):
        for j in range(len(data[i])):
            for x in range(scaler):
                for y in range(scaler):
                    value =(data[i][j]+x+y )%9
                    if value == 0:
                        value = 9
                    tmp[i+x*len(data)][j+y*len(data[i])] = value
    print(tmp)
    return tmp 



In [15]:
if(False):
    map = GetRiskAreaImage2('./data/aoc15_test1.txt', 5)

    startRoute = Route([Node(0,0)], 0)
    explored = np.zeros(map.shape, dtype=np.int32)
    explored += sys.maxsize
    explored[0][0] = 0
    timeStart = time.time()
    route = FindPathWithLowestRisk2(map, Node(0,0), Node(map.shape[0]-1,map.shape[1]-1) , [startRoute], explored)
    timeEnd = time.time()
    MarkedRoute = MarkRoute(map.shape, route)
    print(f'TimeToFind: {timeEnd-timeStart}')
    print(f'Risk: {route.risk}')
    showScaledImage(MarkedRoute, breaktime=1, scaling = 1, name='Result')

    map = GetRiskAreaImage2('./data/aoc15_test1.txt', 10)

    startRoute = Route([Node(0,0)], 0)
    explored = np.zeros(map.shape, dtype=np.int32)
    explored += sys.maxsize
    explored[0][0] = 0
    timeStart = time.time()
    route = FindPathWithLowestRisk2(map, Node(0,0), Node(map.shape[0]-1,map.shape[1]-1) , [startRoute], explored)
    timeEnd = time.time()
    MarkedRoute = MarkRoute(map.shape, route)
    print(f'TimeToFind: {timeEnd-timeStart}')
    print(f'Risk: {route.risk}')
    showScaledImage(MarkedRoute, breaktime=1, scaling = 1, name='Result')

In [16]:
if(False):
    map = GetRiskAreaImage2('./data/aoc15.txt')

    startRoute = Route([Node(0,0)], 0)
    explored = np.zeros(map.shape, dtype=np.int32)
    explored += sys.maxsize
    explored[0][0] = 0
    route = FindPathWithLowestRisk2(map, Node(0,0), Node(map.shape[0]-1,map.shape[1]-1) , [startRoute], explored)

    MarkedRoute = MarkRoute(map.shape, route)

    print(f'Risk: {route.risk}')
    showScaledImage(MarkedRoute, breaktime=0, scaling = 1, name='Result')

In [17]:
class Node:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.value = -1
        self.distance = -1
    def __repr__(self):
        return 'Node({}, {})'.format(self.x, self.y)
    
    

class Route:
    def __init__(self, nodes, risk):
        self.nodes = [x for x in nodes]
        self.risk = risk
    def AddNode(self, node):
        self.nodes.append(node)
        self.distanceToEnd = node.distance
    def Contains(self, node):
        return node in self.nodes
    def __repr__(self):
        return 'Route({}, {})'.format(self.nodes, self.risk)

def GetAdjacentNodes( map, node):
    adjacentNodes = []
    if node.x > 0: adjacentNodes.append(map[node.x-1][ node.y])
    if node.x < len(map)-1: adjacentNodes.append(map[node.x+1][ node.y])
    if node.y > 0: adjacentNodes.append(map[node.x][ node.y-1])
    if node.y < len(map[0])-1: adjacentNodes.append(map[node.x][ node.y+1])
    return adjacentNodes

def FindPathWithLowestRisk3( map, current, end, pathTree,exploredNodesMap, optimalPath = None):
    optimumPathImage = np.zeros(map.shape, dtype=np.uint8)
    while len(pathTree) > 0:    
        currentRoute = pathTree.pop(0)
        current = currentRoute.nodes[-1]
        #print(f"pathTree: {len(pathTree)} node: {current}, value: {current.value}")   
        if currentRoute.risk > current.value and current.value != -1:
            continue

        if optimumPathImage[current.x,current.y] >0:
            collect = False
            for node in optimalPath.nodes:
                if collect == True:
                    currentRoute.AddNode(node)
                    currentRoute.risk += map[node.x,node.y]
                    node.value = currentRoute.risk
                elif current == node:
                    collect = True
            if (optimalPath.risk > currentRoute.risk and currentRoute.nodes[-1] == end):
                optimumPathImage = MarkRoute(map.shape, currentRoute)
                optimalPath = currentRoute
                #showScaledImage(optimumPathImage, breaktime=1, scaling = 5, name='optimalPath')
                continue

        #showScaledImage( MarkRoute(map.shape,currentRoute) , breaktime=1, scaling = 1, name='exploredNodesMap')
        
        # Get the adjacent nodes
        adjacentNodes = GetAdjacentNodes(exploredNodesMap, current)
        for node in adjacentNodes:
            tmpNodeRisk = currentRoute.risk + GetRisk(map, node)
            if (node.value > tmpNodeRisk or node.value == -1) and currentRoute.Contains(node) == False:
                #print(tmpNodeRisk)
                newCurrentRoute = Route(currentRoute.nodes, tmpNodeRisk)
                newCurrentRoute.AddNode(node)
                node.value = tmpNodeRisk
                pathTree.append(newCurrentRoute)
        pathTree.sort( key=lambda x: x.risk)  
        if current == end:
            if optimalPath == None or (optimalPath.risk > currentRoute.risk and currentRoute.nodes[-1] == end):
                optimumPathImage = MarkRoute(map.shape, currentRoute)
                optimalPath = currentRoute
                #showScaledImage(optimumPathImage, breaktime=1, scaling = 1, name='optimalPath')
            
    return optimalPath      

In [18]:
if(True):
    map = GetRiskAreaImage2('./data/aoc15_test1.txt', 5)

    startRoute = Route([Node(0,0)], 0)
    end = Node(map.shape[0]-1,map.shape[1]-1)
    nodeMap = []
    for i in range(0,map.shape[0]):
        nodeMap.append([])
        for j in range(0,map.shape[1]):
            tmp = Node(i,j)
            tmp.distance = math.sqrt(abs(i-end.x)**2 + abs(j-end.y)**2)
            nodeMap[i].append(tmp)



    timeStart = time.time()
    route = FindPathWithLowestRisk3(map, nodeMap[0][0], nodeMap[map.shape[0]-1][map.shape[1]-1] , [startRoute], nodeMap)
    timeEnd = time.time()
    MarkedRoute = MarkRoute(map.shape, route)
    print(f'TimeToFind: {timeEnd-timeStart}')
    print(f'Risk: {route.risk}')
    showScaledImage(MarkedRoute, breaktime=10, scaling = 1, name='Result1')
    showWriteImage(MarkedRoute, scaling = 1, loc= './images/aoc15_result_test.bmp')
if(True):
    map = GetRiskAreaImage2('./data/aoc15.txt', 5)

    startRoute = Route([Node(0,0)], 0)
    end = Node(map.shape[0]-1,map.shape[1]-1)
    nodeMap = []
    for i in range(0,map.shape[0]):
        nodeMap.append([])
        for j in range(0,map.shape[1]):
            tmp = Node(i,j)
            tmp.distance = math.sqrt(abs(i-end.x)**2 + abs(j-end.y)**2)
            nodeMap[i].append(tmp)



    timeStart = time.time()
    route = FindPathWithLowestRisk3(map, nodeMap[0][0], nodeMap[map.shape[0]-1][map.shape[1]-1] , [startRoute], nodeMap)
    timeEnd = time.time()
    MarkedRoute = MarkRoute(map.shape, route)
    print(f'TimeToFind: {timeEnd-timeStart}')
    print(f'Risk: {route.risk}')
    showScaledImage(MarkedRoute, breaktime=10, scaling = 1, name='Result1')
    showWriteImage(MarkedRoute, scaling = 1, loc= './images/aoc15_result.bmp')

[[1 1 6 ... 2 8 6]
 [1 3 8 ... 1 2 6]
 [2 1 3 ... 7 6 3]
 ...
 [7 5 6 ... 5 2 8]
 [5 6 4 ... 4 1 9]
 [6 7 5 ... 4 7 9]]
TimeToFind: 0.0539708137512207
Risk: 315
[[4 7 6 ... 2 8 4]
 [2 8 7 ... 3 3 1]
 [4 6 8 ... 4 8 6]
 ...
 [2 2 9 ... 5 6 7]
 [4 9 4 ... 8 8 4]
 [8 3 1 ... 8 2 8]]
TimeToFind: 28.29026460647583
Risk: 3002
